# QQuill (SpellBook) Export to Markdown

This notebook gives a very easy way to convert a SpellBook (the one released before 2023) export to a bunch of markdown files, including the conversion of created JPGs to their original format and designation.

First specify the variables in the setup section, then run each cell sequentially in python 3.9+. You can choose to include ephemeris calculation and data in the markdown files.

In doubt, run your questions by ChatGPT or your favorite AI,, and if needed contact `adam@blvckstudios.com` directly.

In [7]:
import json
import os

In [8]:
import json

# Path to your JSON file
file_path = './data/SpellBook_Backup_2024-02-10.json'

# Open and read the JSON file
with open(file_path, 'r') as file:
    data = json.load(file)

# Access the desired object and assign it to MARKS
MARKS = data['state']['things']['all']

# You can now use the MARKS variable as needed
print(MARKS.keys())

MARKS['20210328194335']


dict_keys(['20210328194335', '20210329133009', '20210329175748', '20210409202610', '20210410012020', '20210410012718', '20210410193137', '20210411005714', '20210412112137', '20210412232449', '20210412234543', '20210414105607', '20210414195400', '20210416000207', '20210416013505', '20210416142336', '20210418081815', '20210418082942', '20210418215434', '20210418225441', '20210419001153', '20210419145815', '20210420020816', '20210420215242', '20210420215438', '20210420215647', '20210420220719', '20210420221915', '20210420221955', '20210420222929', '20210420224803', '20210422010933', '20210422011557', '20210422111855', '20210423012949', '20210423153104', '20210423160417', '20210423203921', '20210424183133', '20210425033500', '20210425033832', '20210426125612', '20210427083513', '20210427101853', '20210428063812', '20210430122427', '20210501130032', '20210501210303', '20210501215705', '20210501232715', '20210502141606', '20210502161611', '20210502171933', '20210502173026', '20210503192815',

{'id': '20210328194335',
 'title': 'Meaning of 🜁🜂🜄🜃',
 'content': 'Here’s the authentic meaning of 🜁🜂🜄🜃:\nי - choices at hand\nה - envisioning and choosing one thing \nו - feeling the thing into reality\nה - seeing the chosen object through physical eye ',
 'pictures': {'photos': {}, 'photoIds': []},
 'creation_dt': '"2021-03-28T17:43:35.646Z"',
 'modified_dt': '"2021-04-23T02:18:04.411Z"',
 'elements': {},
 'elements_selected': [],
 'elements_row': [],
 'el_tree': {'root': []},
 'el_ids': [],
 'el_depth': {'root': 0},
 'el_parents': {'root': None},
 'el_num_child': {'root': 0},
 'groups': [],
 'counts': {'ups': 0,
  'downs': 0,
  'air': 0,
  'fire': 0,
  'water': 0,
  'earth': 0,
  'charge_remaining': 0,
  'last_charge_dt': '"2021-08-31T22:00:20.949Z"',
  'charge_history': {'"2021-03-28T17:43:35.646Z"': 0,
   '"2021-08-31T22:00:20.949Z"': 0}},
 'ephemeris': {'sun': {'rad': 0.1592048509481258,
   'hex': {'hex': 17, 'lin': 5, 'col': 4, 'ton': 5}},
  'mercury': {'rad': 6.106745377591213,

In [9]:
def addTabs(text, depth):
    # Split the text into lines
    lines = text.split("\n")
    # Indent each line with the specified number of tabs
    indentedLines = ['\t' * depth + line for line in lines]
    # Join the lines back together into a string
    indentedText = '\n'.join(indentedLines)
    return indentedText

def getMDElementTree(thing):
    elements = thing['elements']
    elements_selected = thing['elements_selected']
    elements_row = thing['elements_row']
    el_tree = thing['el_tree']
    el_ids = thing['el_ids']
    el_depth = thing['el_depth']
    el_parents = thing['el_parents']
    el_num_child = thing['el_num_child']

    # sortedElements = thing['elements']  # Assuming sortedElements is part of 'thing'
    # print(sortedElements)

    element_section_string = "## Element Section\n\n"

    for kk, el in thing['elements'].items():
        depth = thing['el_depth'][kk]
        element = el['element']
        content = elements[el['id']]['content']

        for j in range(depth):
            element_section_string += '\t'
        element_section_string += '- ' + element + '\n'
        element_section_string += addTabs(content, depth + 1)  # Assuming addTabs is a defined function
        element_section_string += '\n'
    return element_section_string

In [10]:
import yaml

def generate_yaml_section(obj, fields):
    yaml_section = "---\n"
    for field in fields:
        if field in obj:
            # Check if the field value is a dictionary (nested object)
            if isinstance(obj[field], dict):
                # Recursively generate YAML for the nested object
                nested_yaml = yaml.dump({field: obj[field]}, default_flow_style=False)
                yaml_section += nested_yaml
            else:
                yaml_section += f"{field}: {obj[field]}\n"
    yaml_section += "---\n"
    return yaml_section

In [11]:
import json

output_dir = './qquill'

from datetime import datetime

def convert_to_iso(date_string):
    # Parse the date string into a datetime object
    date = datetime.strptime(date_string, '%Y%m%d%H%M%S')
    
    # Convert the datetime object to an ISO string
    iso_string = date.isoformat()
    
    return iso_string

def process_marks_and_save(marks_dict, fields):
    # Ensure the directory exists
    output_dir = './qquil/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Iterate through each key in the MARKS dictionary
    for key in marks_dict:
        obj = marks_dict[key]

        # Create a new object with the required fields
        g = getMDElementTree(obj)

        # Convert the key to an ISO date
        iso_date = convert_to_iso(key)

        # Generate YAML section
        yaml_section = generate_yaml_section(obj, fields)

        # Compose simple markdown with YAML section
        markdown = f"{yaml_section}\n# {obj['title']}\n{iso_date}\n\n{obj['content']}\n\n{g}"

        # Define the output file path using the object's ID or any unique identifier
        output_file_path = os.path.join(output_dir, f"{obj['id']}.md")
        
        # Save the new object to a new markdown file
        with open(output_file_path, 'w') as outfile:
            outfile.write(markdown)



In [12]:
process_marks_and_save(MARKS, ['id', 'thing_element', 'ephemeris'])